In [1]:
import pandas as pd
from helper.onemap import OneMapQuery, convert_second_to_time_with_s
from vroom import Vehicle, Shipment, ShipmentStep, Input, TimeWindow
import time
from datetime import datetime
import re
from tqdm import tqdm
import folium

In [2]:
df = pd.read_excel('store/data/pickup_delivery_jobs.xlsx')
df

,job_id,pickup_address,delivery_address,pickup_time_window,delivery_time_window
0,1,38 TOA PAYOH RISE SINGAPORE 298113,473 RIVER VALLEY ROAD VALLEY PARK SINGAPORE 24...,"['09:39:00', '13:19:00']","['20:01:00', '20:44:00']"
1,2,81 WEST COAST DRIVE HUNDRED TREES SINGAPORE 12...,5 SCIENCE PARK DRIVE GENEO SINGAPORE 118265,"['13:29:00', '15:08:00']","['17:18:00', '19:33:00']"
2,3,55 KEONG SAIK ROAD KESA HOUSE SINGAPORE 089158,10 LORONG 26 GEYLANG GUILIN COURT SINGAPORE 39...,"['16:09:00', '17:12:00']","['16:25:00', '18:55:00']"
3,4,102 TANAH MERAH BESAR ROAD COMPASS GREEN SINGA...,17 CHANGI SOUTH STREET 2 SINGAPORE 486129,"['21:42:00', '21:53:00']","['21:47:00', '21:48:00']"
4,5,7 JALAN KEMAMAN TAI PEI YUEN TEMPLE SINGAPORE ...,473 RIVER VALLEY ROAD VALLEY PARK SINGAPORE 24...,"['14:44:00', '20:09:00']","['21:14:00', '21:37:00']"
5,6,7 ARMENIAN STREET ALL SAINTS PRESBYTERIAN CHUR...,20 SAINT MICHAEL'S ROAD SUNVILLE SINGAPORE 327979,"['13:03:00', '16:37:00']","['18:26:00', '19:34:00']"
6,7,1003 LOWER DELTA ROAD TERESA VILLE SINGAPORE 0...,211 BEDOK ROAD MAN FATT LAM BUDDHIST TEMPLE SI...,"['17:50:00', '18:33:00']","['17:55:00', '20:51:00']"
7,8,7 TANJONG RHU ROAD THE WATERSIDE SINGAPORE 436887,10 SCOTTS ROAD GRAND HYATT SINGAPORE SINGAPORE...,"['18:41:00', '20:04:00']","['21:23:00', '21:26:00']"
8,9,3 MOUNT EMILY ROAD EMILY RESIDENCE SINGAPORE 2...,120 MAXWELL ROAD ATM CITIBANK TANJONG PAGAR SM...,"['20:44:00', '21:24:00']","['21:41:00', '21:44:00']"
9,10,17 ENG HOON STREET ENG HOON MANSIONS SINGAPORE...,6 CHANGI NORTH STREET 1 SPANNERS BLDG SINGAPOR...,"['09:43:00', '21:19:00']","['14:12:00', '19:17:00']"


In [3]:
# Cell 3: Convert time windows to seconds
def convert_time_window_to_seconds(time_window):
    """Convert time window string "['HH:MM:SS', 'HH:MM:SS']" to seconds"""
    if isinstance(time_window, str):
        # Remove brackets and split the string
        time_window = time_window.strip("[]").replace("'", "").split(", ")
    if isinstance(time_window, list) and len(time_window) == 2:
        start_time = datetime.strptime(time_window[0], '%H:%M:%S')
        end_time = datetime.strptime(time_window[1], '%H:%M:%S')
        start_seconds = start_time.hour * 3600 + start_time.minute * 60 + start_time.second
        end_seconds = end_time.hour * 3600 + end_time.minute * 60 + end_time.second
        return start_seconds, end_seconds
    return None, None

# Convert pickup and delivery time windows to seconds
df['pickup_time_window_seconds'] = df['pickup_time_window'].apply(convert_time_window_to_seconds)
df['delivery_time_window_seconds'] = df['delivery_time_window'].apply(convert_time_window_to_seconds)
df

,job_id,pickup_address,delivery_address,pickup_time_window,delivery_time_window,pickup_time_window_seconds,delivery_time_window_seconds
0,1,38 TOA PAYOH RISE SINGAPORE 298113,473 RIVER VALLEY ROAD VALLEY PARK SINGAPORE 24...,"['09:39:00', '13:19:00']","['20:01:00', '20:44:00']","(34740, 47940)","(72060, 74640)"
1,2,81 WEST COAST DRIVE HUNDRED TREES SINGAPORE 12...,5 SCIENCE PARK DRIVE GENEO SINGAPORE 118265,"['13:29:00', '15:08:00']","['17:18:00', '19:33:00']","(48540, 54480)","(62280, 70380)"
2,3,55 KEONG SAIK ROAD KESA HOUSE SINGAPORE 089158,10 LORONG 26 GEYLANG GUILIN COURT SINGAPORE 39...,"['16:09:00', '17:12:00']","['16:25:00', '18:55:00']","(58140, 61920)","(59100, 68100)"
3,4,102 TANAH MERAH BESAR ROAD COMPASS GREEN SINGA...,17 CHANGI SOUTH STREET 2 SINGAPORE 486129,"['21:42:00', '21:53:00']","['21:47:00', '21:48:00']","(78120, 78780)","(78420, 78480)"
4,5,7 JALAN KEMAMAN TAI PEI YUEN TEMPLE SINGAPORE ...,473 RIVER VALLEY ROAD VALLEY PARK SINGAPORE 24...,"['14:44:00', '20:09:00']","['21:14:00', '21:37:00']","(53040, 72540)","(76440, 77820)"
5,6,7 ARMENIAN STREET ALL SAINTS PRESBYTERIAN CHUR...,20 SAINT MICHAEL'S ROAD SUNVILLE SINGAPORE 327979,"['13:03:00', '16:37:00']","['18:26:00', '19:34:00']","(46980, 59820)","(66360, 70440)"
6,7,1003 LOWER DELTA ROAD TERESA VILLE SINGAPORE 0...,211 BEDOK ROAD MAN FATT LAM BUDDHIST TEMPLE SI...,"['17:50:00', '18:33:00']","['17:55:00', '20:51:00']","(64200, 66780)","(64500, 75060)"
7,8,7 TANJONG RHU ROAD THE WATERSIDE SINGAPORE 436887,10 SCOTTS ROAD GRAND HYATT SINGAPORE SINGAPORE...,"['18:41:00', '20:04:00']","['21:23:00', '21:26:00']","(67260, 72240)","(76980, 77160)"
8,9,3 MOUNT EMILY ROAD EMILY RESIDENCE SINGAPORE 2...,120 MAXWELL ROAD ATM CITIBANK TANJONG PAGAR SM...,"['20:44:00', '21:24:00']","['21:41:00', '21:44:00']","(74640, 77040)","(78060, 78240)"
9,10,17 ENG HOON STREET ENG HOON MANSIONS SINGAPORE...,6 CHANGI NORTH STREET 1 SPANNERS BLDG SINGAPOR...,"['09:43:00', '21:19:00']","['14:12:00', '19:17:00']","(34980, 76740)","(51120, 69420)"


In [4]:
# Cell 4: Initialize OneMapQuery and process locations
om = OneMapQuery()
depot_latlong = (1.31158791334051, 103.863375124429)  # CT Hub 2
location_map = [depot_latlong]  # Start with depot

print("Processing locations...")
for idx, row in tqdm(df.iterrows()):
    pickup_address = row['pickup_address']
    delivery_address = row['delivery_address']
    
    # Process pickup address
    pickup_postal_match = re.search(r'SINGAPORE (\d{6})', pickup_address)
    if pickup_postal_match:
        pickup_postal = pickup_postal_match.group(1)
        pickup_latlong = om.get_postal_latlong(pickup_postal)
        if pickup_latlong and pickup_latlong not in location_map:
            location_map.append(pickup_latlong)
    
    # Process delivery address
    delivery_postal_match = re.search(r'SINGAPORE (\d{6})', delivery_address)
    if delivery_postal_match:
        delivery_postal = delivery_postal_match.group(1)
        delivery_latlong = om.get_postal_latlong(delivery_postal)
        if delivery_latlong and delivery_latlong not in location_map:
            location_map.append(delivery_latlong)
    time.sleep(0.5)

print(f"\nUnique locations processed: {len(location_map)-1}")

df

Processing locations...


30it [00:40,  1.33s/it]


Unique locations processed: 58


,job_id,pickup_address,delivery_address,pickup_time_window,delivery_time_window,pickup_time_window_seconds,delivery_time_window_seconds
0,1,38 TOA PAYOH RISE SINGAPORE 298113,473 RIVER VALLEY ROAD VALLEY PARK SINGAPORE 24...,"['09:39:00', '13:19:00']","['20:01:00', '20:44:00']","(34740, 47940)","(72060, 74640)"
1,2,81 WEST COAST DRIVE HUNDRED TREES SINGAPORE 12...,5 SCIENCE PARK DRIVE GENEO SINGAPORE 118265,"['13:29:00', '15:08:00']","['17:18:00', '19:33:00']","(48540, 54480)","(62280, 70380)"
2,3,55 KEONG SAIK ROAD KESA HOUSE SINGAPORE 089158,10 LORONG 26 GEYLANG GUILIN COURT SINGAPORE 39...,"['16:09:00', '17:12:00']","['16:25:00', '18:55:00']","(58140, 61920)","(59100, 68100)"
3,4,102 TANAH MERAH BESAR ROAD COMPASS GREEN SINGA...,17 CHANGI SOUTH STREET 2 SINGAPORE 486129,"['21:42:00', '21:53:00']","['21:47:00', '21:48:00']","(78120, 78780)","(78420, 78480)"
4,5,7 JALAN KEMAMAN TAI PEI YUEN TEMPLE SINGAPORE ...,473 RIVER VALLEY ROAD VALLEY PARK SINGAPORE 24...,"['14:44:00', '20:09:00']","['21:14:00', '21:37:00']","(53040, 72540)","(76440, 77820)"
5,6,7 ARMENIAN STREET ALL SAINTS PRESBYTERIAN CHUR...,20 SAINT MICHAEL'S ROAD SUNVILLE SINGAPORE 327979,"['13:03:00', '16:37:00']","['18:26:00', '19:34:00']","(46980, 59820)","(66360, 70440)"
6,7,1003 LOWER DELTA ROAD TERESA VILLE SINGAPORE 0...,211 BEDOK ROAD MAN FATT LAM BUDDHIST TEMPLE SI...,"['17:50:00', '18:33:00']","['17:55:00', '20:51:00']","(64200, 66780)","(64500, 75060)"
7,8,7 TANJONG RHU ROAD THE WATERSIDE SINGAPORE 436887,10 SCOTTS ROAD GRAND HYATT SINGAPORE SINGAPORE...,"['18:41:00', '20:04:00']","['21:23:00', '21:26:00']","(67260, 72240)","(76980, 77160)"
8,9,3 MOUNT EMILY ROAD EMILY RESIDENCE SINGAPORE 2...,120 MAXWELL ROAD ATM CITIBANK TANJONG PAGAR SM...,"['20:44:00', '21:24:00']","['21:41:00', '21:44:00']","(74640, 77040)","(78060, 78240)"
9,10,17 ENG HOON STREET ENG HOON MANSIONS SINGAPORE...,6 CHANGI NORTH STREET 1 SPANNERS BLDG SINGAPOR...,"['09:43:00', '21:19:00']","['14:12:00', '19:17:00']","(34980, 76740)","(51120, 69420)"


In [ ]:
# Cell 5: Calculate route matrices
print("\nCalculating route matrices...")
duration_matrix, distance_matrix = om.get_route_matrices(location_map)


Calculating route matrices...
Calculating routes for 41 new locations...


 26%|██▌       | 1202/4592 [22:09<1:00:37,  1.07s/it]

In [ ]:
# Cell 5: Get vehicle information
def get_time_in_seconds(time_str):
    """Convert HH:MM format to seconds"""
    try:
        hours, minutes = map(int, time_str.split(':'))
        return hours * 3600 + minutes * 60
    except:
        raise ValueError("Time must be in HH:MM format")

def get_vehicle_time_window(vehicle_num=None):
    """Get start and end time for a vehicle"""
    while True:
        try:
            vehicle_prefix = f" for Vehicle {vehicle_num}" if vehicle_num else ""
            start_time = input(f"Enter start time (HH:MM){vehicle_prefix}: ")
            end_time = input(f"Enter end time (HH:MM){vehicle_prefix}: ")
            start_seconds = get_time_in_seconds(start_time)
            end_seconds = get_time_in_seconds(end_time)
            if end_seconds <= start_seconds:
                print("Error: End time must be after start time")
                continue
            return start_seconds, end_seconds
        except ValueError as e:
            print(f"Error: {e}")

# Get number of vehicles
num_vehicles = int(input("Enter number of vehicles: "))

# Ask if all vehicles have same time window
same_time_window = input("Do all vehicles have the same time window? (yes/no): ").lower() == 'yes'

# Get time windows for vehicles
vehicle_time_windows = []
if same_time_window:
    print("\nEntering time window for ALL vehicles:")
    start_seconds, end_seconds = get_vehicle_time_window()
    vehicle_time_windows = [(start_seconds, end_seconds)] * num_vehicles
else:
    print("\nEntering individual time windows:")
    for i in range(num_vehicles):
        start_seconds, end_seconds = get_vehicle_time_window(i+1)
        vehicle_time_windows.append((start_seconds, end_seconds))

In [ ]:
# Cell 6: Create VROOM input
vroom_input = Input()

# Add vehicles with time windows
for i in range(num_vehicles):
    vehicle = Vehicle(
        id=i+1,
        start=0,
        end=0,
        time_window=TimeWindow(
            start=vehicle_time_windows[i][0],
            end=vehicle_time_windows[i][1]
        )
    )
    vroom_input.add_vehicle(vehicle)

# Add shipments (pickup and delivery pairs)
for idx, row in df.iterrows():
    # Get coordinates
    pickup_postal = re.search(r'SINGAPORE (\d{6})', row['pickup_address']).group(1)
    delivery_postal = re.search(r'SINGAPORE (\d{6})', row['delivery_address']).group(1)
    pickup_coord = om.get_postal_latlong(pickup_postal)
    delivery_coord = om.get_postal_latlong(delivery_postal)
    
    # Create pickup step
    pickup_start, pickup_end = row['pickup_time_window_seconds']
    pickup = ShipmentStep(
        id=row['job_id'] * 10 + 1,
        location=location_map.index(pickup_coord),
        time_windows=[TimeWindow(start=pickup_start, end=pickup_end)]
    )
    
    # Create delivery step
    delivery_start, delivery_end = row['delivery_time_window_seconds']
    delivery = ShipmentStep(
        id=row['job_id'] * 10 + 2,
        location=location_map.index(delivery_coord),
        time_windows=[TimeWindow(start=delivery_start, end=delivery_end)]
    )
    
    # Add shipment directly with pickup and delivery steps
    vroom_input.add_shipment(pickup=pickup, delivery=delivery)

In [ ]:
# Cell 7: Set matrices and solve
vroom_input.set_durations_matrix(
    profile="car",
    matrix_input=duration_matrix
)
vroom_input.set_distances_matrix(
    profile="car",
    matrix_input=distance_matrix
)

solution = vroom_input.solve(exploration_level=5, nb_threads=4)


In [ ]:
# Cell 8: Convert solution routes to a dataframe for easier analysis
route_df = solution.routes.copy()

print("\nOptimization Results:")
print(f"Total duration: {convert_second_to_time_with_s(solution.summary.duration)}")
print(f"Total distance: {solution.summary.distance} meters")

In [ ]:
import folium

# Create a map centered on Singapore
m = folium.Map(location=[1.352083, 103.819839], zoom_start=12, tiles="cartodbpositron")

# Add depot marker with special styling
folium.CircleMarker(
    location=depot_latlong,
    popup='Depot (CT Hub 2)',
    tooltip='Depot',
    color='red',
    fill=True,
    fillColor='red',
    radius=10
).add_to(m)

In [ ]:
# Group routes by vehicle
for vehicle_id in range(1, num_vehicles + 1):
    vehicle_route = route_df[route_df['vehicle_id'] == vehicle_id]
    route_sequence = []
    
    # Get sequence of locations for this vehicle
    for step in vehicle_route.itertuples():
        route_sequence.append(location_map[step.location_index])
    
    # Plot routes between consecutive points
    for i in range(len(route_sequence)-1):
        start = route_sequence[i]
        end = route_sequence[i+1]
        # Plot route with sequence numbers, using vehicle_id as color index
        om.plot_routes(start, end, m, vehicle_id, (i, i+1))

In [ ]:
m

In [ ]:
m.save('pickup_delivery.html')

# Get all job IDs from the solution (excluding start/end rows)
completed_jobs = route_df[route_df['type'] == 'job']['id'].tolist()

# Get all job IDs from our original dataframe
all_jobs = df['job_id'].tolist()

# Check for missing jobs
missing_jobs = set(all_jobs) - set(completed_jobs)

if len(missing_jobs) == 0:
    print("All jobs were fulfilled!")
else:
    print("Warning: Some jobs were not fulfilled!")
    print("Missing jobs:", sorted(list(missing_jobs)))
    
print("\nMap has been saved as 'optimized_route_with_time.html'")
print(f"Total vehicles used: {len(route_df['vehicle_id'].unique())}")
print(f"Total stops: {len(completed_jobs)}")

In [ ]:
print("\nRoute Details per Vehicle:")
for vehicle_id in range(1, num_vehicles + 1):
    vehicle_route = route_df[route_df['vehicle_id'] == vehicle_id]
    if not vehicle_route.empty:
        print(f"\nVehicle {vehicle_id}:")
        print(f"Start time: {convert_second_to_time_with_s(vehicle_route.iloc[0]['arrival'])}")
        print(f"End time: {convert_second_to_time_with_s(vehicle_route.iloc[-1]['arrival'])}")
        print(f"Total duration: {convert_second_to_time_with_s(vehicle_route.iloc[-1]['duration'])}")
        print(f"Total distance: {vehicle_route.iloc[-1]['distance']} meters")
        print(f"Number of stops: {len(vehicle_route[vehicle_route['type'] == 'job'])}")